# Tree-of-Life (L-shape) viewer with Cosmograph

This notebook loads one or more Newick trees (either from a **.tgz/.tar.gz** archive containing multiple files or from a single **.newick/.nwk/.tre/.tree/.gz** file), parses them with BioPython, computes a **T-shaped rectangular phylogram layout**, and renders it as a **Cosmograph** widget.

**Guaranteed by this layout:**
- **L-shape edges:** For every edge, we draw a horizontal segment (weighted by branch length) and then a vertical segment to the child; this is achieved by inserting “bend” nodes.
- **All leaves aligned on one vertical line** (a right margin): terminal branches are extended horizontally (drawn as extra horizontal “tip” segments) so leaves land exactly on the same X.
- **No point overlap** (within a tree): leaves are placed on evenly spaced Y levels; “bend” nodes are slightly de-overlapped if siblings happen to have identical branch lengths.
- **Leaves are colored differently** from other nodes.
- **Multiple trees** (if provided) are **stacked vertically** (non-overlapping bands), each tree rebased to start at X=0.
- **All nodes & links are displayed** with **no sampling** (suitable for very large trees; turn off simulation and sampling in Cosmograph).

> Tip: Cosmograph needs WebGL. In JupyterLab/VS Code, ensure hardware acceleration is enabled.

# 1) Setup, imports, global config


In [1]:
import os
import re
import math
import sys
import gc
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Iterable

import numpy as np
import pandas as pd
from IPython.display import display

# ---------- Debug logger (stderr) ----------
def log(*a, **k):
    print(*a, **k, file=sys.stderr, flush=True)

# ---------- Input ----------
# Set this to your .newick/.nwk/.tre file OR paste a Newick string.
INPUT_PATH = "/Users/gushchin_a/PyCharmProjects/Gtol/opentree15.1_tree/labelled_supertree/labelled_supertree_ottnames.tre"  # change if needed

# ---------- Layout controls ----------
LEAF_Y_STEP     = 4.0   # constant vertical spacing between leaves (compact but readable)
PARENT_STUB_LEN = 6.0   # constant horizontal stub from a parent to its vertical connector
TIP_ALIGN_PAD   = 20.0  # distance to the right of the furthest leaf x where tip markers are placed
EPS             = 1e-9  # numeric safety for comparisons

# ---------- Colors (good contrast on default Cosmograph dark bg) ----------
COLOR_LEAF     = "#f5d76e"  # warm yellow
COLOR_INTERNAL = "#8ab4f8"  # light blue
COLOR_BEND     = "#9aa0a6"  # gray for technical bend/junction nodes
COLOR_TIP      = "#bdbdbd"  # pale gray for right-side tip markers (unconnected)

# ---------- Visual constants for the widget (can be tweaked in Render cell too) ----------
POINT_SIZE     = 6.0
LINK_WIDTH     = 1.4
STATIC_LABELS  = False
DYNAMIC_LABELS = True
HOVER_LABELS   = True

log("[Init] Notebook config loaded.")


[Init] Notebook config loaded.


# 2) Input preview (sanity)


In [2]:
def preview_input(path_or_text: str, n_chars: int = 400):
    if os.path.exists(path_or_text):
        size = os.path.getsize(path_or_text)
        with open(path_or_text, "r", encoding="utf-8") as f:
            head = f.read(n_chars)
        log(f"[Input] file={path_or_text!r}  size={size/1e6:.2f} MB  preview(len={len(head)})")
        print(head.replace("\n", " ")[:n_chars])
    else:
        s = str(path_or_text)
        s = s[:n_chars]
        log(f"[Input] raw string provided, preview(len={len(s)})")
        print(s.replace("\n"," "))

preview_input(INPUT_PATH)


[Input] file='/Users/gushchin_a/PyCharmProjects/Gtol/opentree15.1_tree/labelled_supertree/labelled_supertree_ottnames.tre'  size=85.83 MB  preview(len=400)


((((((((((((((((((((((((((((((((((((((((((((Elaeocarpus_williamsianus_ott2,((Platytheca_galioides_ott4,Platytheca_verticillata_ott414964,Platytheca_anasima_ott6107258,Platytheca_juniperina_ott6107259)Platytheca_ott414967,((((((((((((Elaeocarpus_foveolatus_ott6,(Elaeocarpus_reticulatus_ott22687,Elaeocarpus_bancroftii_ott471450)mrcaott22687ott471450)mrcaott6ott22687,Elaeocarpus_kirtonii_ott16)mrcaot


# 3) Robust Newick parser (names + branch lengths, multiple trees)

In [3]:
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Iterable
import re
import sys

def log(*a, **k):
    print(*a, **k, file=sys.stderr, flush=True)

@dataclass
class TNode:
    id: str
    name: str = ""
    parent: Optional[str] = None
    blen: float = 0.0                 # branch length from parent to this node
    children: List[str] = field(default_factory=list)

# Strip simple [ ... ] comments (no nesting in most Newick files)
def _strip_newick_comments(s: str) -> str:
    return re.sub(r"\[[^\]]*]", "", s)

# If INPUT_PATH is a file with many trees separated by ';', yield them
def iter_newick_streams(path_or_text: str) -> Iterable[Tuple[str, str]]:
    import os
    if os.path.exists(path_or_text):
        with open(path_or_text, "r", encoding="utf-8") as f:
            txt = f.read()
        txt = _strip_newick_comments(txt).strip()
        parts = [p.strip() for p in txt.split(";") if p.strip()]
        if not parts:
            raise ValueError("[Newick] No trees found in file.")
        for i, p in enumerate(parts):
            yield (f"tree{i}", p + ";")
    else:
        s = _strip_newick_comments(str(path_or_text)).strip()
        yield ("tree0", s if s.endswith(";") else s + ";")

# Tokenizer for (),:; or labels (unquoted)
_TOKEN_RE = re.compile(r"\s*([(),:;])\s*|\s*([^(),:;]+)")

def _tokenize(s: str):
    i = 0
    L = len(s)
    while i < L:
        m = _TOKEN_RE.match(s, i)
        if not m:
            i += 1
            continue
        i = m.end()
        yield m.group(1) if m.group(1) is not None else m.group(2)

def parse_newick(s: str) -> Dict[str, TNode]:
    """Parse a single Newick string into a dict of nodes keyed by id."""
    stack: List[str] = []
    nodes: Dict[str, TNode] = {}
    nid = 0

    def new_id() -> str:
        nonlocal nid
        nid += 1
        return f"n{nid}"

    def add_internal(parent: Optional[str]) -> str:
        u = new_id()
        nodes[u] = TNode(id=u)
        if parent is not None:
            nodes[parent].children.append(u)
            nodes[u].parent = parent
        return u

    def add_leaf(parent: Optional[str], name: str) -> str:
        u = new_id()
        nodes[u] = TNode(id=u, name=name, parent=parent, blen=0.0)
        if parent is not None:
            nodes[parent].children.append(u)
        return u

    last: Optional[str] = None
    expect_len = False

    for tok in _tokenize(s):
        if tok == "(":
            parent = stack[-1] if stack else None
            u = add_internal(parent)
            stack.append(u)
            # We have just opened a new clade – no current node for branch lengths yet.
            last = None

        elif tok == ",":
            # Next child begins; no current 'last' yet
            last = None

        elif tok == ")":
            # Close current clade and set 'last' to this internal node
            if not stack:
                raise ValueError("[Newick] Unbalanced ')'")
            last = stack.pop()

        elif tok == ":":
            # IMPORTANT FIX:
            # If there's a colon but no current node yet (e.g., ',:0.1'),
            # create an implicit unlabeled leaf under the current parent.
            if last is None:
                if not stack:
                    raise ValueError("[Newick] Branch length at top-level with no node.")
                parent = stack[-1]
                last = add_leaf(parent, "")
            expect_len = True

        elif tok == ";":
            break

        else:
            # label or length
            if expect_len:
                # Previous token was ':' → this must be a numeric length
                if last is None:
                    if not stack:
                        raise ValueError("[Newick] Branch length without preceding node.")
                    parent = stack[-1]
                    last = add_leaf(parent, "")
                try:
                    nodes[last].blen = float(tok)
                except ValueError:
                    nodes[last].blen = 0.0
                expect_len = False
            else:
                # token is a label; attach to 'last' if it exists, else it's a new leaf
                if last is None:
                    parent = stack[-1] if stack else None
                    last = add_leaf(parent, tok)
                else:
                    nodes[last].name = tok

    # Handle multiple independent roots by grafting a new artificial root
    roots = [nid for nid, v in nodes.items() if v.parent is None]
    if not roots:
        raise ValueError("[Newick] No root detected.")
    if len(roots) > 1:
        root_id = "root0"
        nodes[root_id] = TNode(id=root_id, name="root", parent=None, blen=0.0, children=roots)
        for r in roots:
            nodes[r].parent = root_id

    log(f"[Parse] nodes={len(nodes):,}, leaves={sum(1 for v in nodes.values() if not v.children):,}")
    return nodes


# 4) Weighted rectangular layout (no crossings, equal leaf spacing, aligned tip markers)


In [4]:
@dataclass
class LayoutResult:
    nodes_df: pd.DataFrame
    links_df: pd.DataFrame

def layout_weighted_rectangular(nodes: Dict[str, TNode],
                                leaf_step: float = LEAF_Y_STEP,
                                parent_stub: float = PARENT_STUB_LEN,
                                tip_pad: float = TIP_ALIGN_PAD) -> LayoutResult:
    """
    Orthogonal (L-shape) layout with:
      - x(u) = cumulative branch length from root to u (weights preserved horizontally),
      - y(leaves) = i * leaf_step (equal spacing), y(internal) = mean(child y),
      - technical 'bend' nodes:
           parent --(constant stub)--> bend_top ==(vertical)==> bend_bot --(residual)--> child
        so stub + residual = branch length; vertical carries no 'distance' (tech nodes ignored).
      - unconnected 'tip' markers placed at a single x to the right of all leaves, at each leaf's y.
    Guarantees no crossings (subtree ordering).
    """
    # --- find the root ---
    root = None
    for k, v in nodes.items():
        if v.parent is None:
            root = k
            break
    if root is None:
        raise ValueError("[Layout] Root not found.")

    # --- build children lists (already there), compute a deterministic child order for no crossings ---
    min_label_cache: Dict[str, str] = {}

    def min_leaf_label(u: str) -> str:
        if u in min_label_cache:
            return min_label_cache[u]
        node = nodes[u]
        if not node.children:
            label = node.name or node.id
        else:
            label = min(min_leaf_label(c) for c in node.children)
        min_label_cache[u] = label
        return label

    for u, v in nodes.items():
        if v.children:
            v.children.sort(key=min_leaf_label)

    # --- assign y to leaves in order; y(internal) = mean(child y) ---
    y_of: Dict[str, float] = {}
    leaves: List[str] = [k for k, v in nodes.items() if not v.children]

    order: List[str] = []
    def dfs_collect(u: str):
        ch = nodes[u].children
        if not ch:
            order.append(u)
        else:
            for c in ch:
                dfs_collect(c)
    dfs_collect(root)

    for i, leaf in enumerate(order):
        y_of[leaf] = i * leaf_step

    def set_internal_y(u: str) -> float:
        ch = nodes[u].children
        if not ch:
            return y_of[u]
        ys = [set_internal_y(c) for c in ch]
        y_of[u] = float(np.mean(ys))
        return y_of[u]
    set_internal_y(root)

    # --- compute x = cumulative branch length from root (weights preserved) ---
    x_of: Dict[str, float] = {root: 0.0}
    stack = [root]
    while stack:
        u = stack.pop()
        for c in nodes[u].children:
            x_of[c] = x_of[u] + max(nodes[c].blen, 0.0)
            stack.append(c)

    # --- aligned tip x (unconnected markers) ---
    max_leaf_x = max((x_of[l] for l in leaves), default=0.0)
    x_tip = max_leaf_x + tip_pad

    # --- build cosmograph nodes/links with technical bends to draw orthogonal edges ---
    node_rows: List[Tuple[str, float, float, str, str, str]] = []  # id, x, y, label, kind, color
    link_rows: List[Tuple[str, str]] = []                          # source_id, target_id

    def add_node(_id: str, x: float, y: float, label: str, kind: str, color: str):
        node_rows.append((_id, float(x), float(y), label, kind, color))

    def add_link(a: str, b: str):
        link_rows.append((a, b))

    for uid, v in nodes.items():
        label = v.name if v.name else uid
        kind  = "internal" if v.children else "leaf"
        color = COLOR_INTERNAL if v.children else COLOR_LEAF
        add_node(uid, x_of[uid], y_of[uid], label, kind, color)

    for u, v in nodes.items():
        if not v.children:
            continue
        xu, yu = x_of[u], y_of[u]
        bends_for_u: List[Tuple[str, float]] = []

        for c in v.children:
            blen = max(nodes[c].blen, 0.0)
            stub = min(parent_stub, blen - EPS) if blen > EPS else 0.0
            xb_top, yb_top = xu + stub, yu
            xb_bot, yb_bot = xb_top, y_of[c]

            bend_top_id = f"{u}__to__{c}__bend_top"
            bend_bot_id = f"{u}__to__{c}__bend_bot"

            add_node(bend_top_id, xb_top, yb_top, "", "bend", COLOR_BEND)
            add_node(bend_bot_id, xb_bot, yb_bot, "", "bend", COLOR_BEND)

            add_link(u, bend_top_id)
            add_link(bend_top_id, bend_bot_id)
            add_link(bend_bot_id, c)

            bends_for_u.append((bend_bot_id, yb_bot))

        bends_for_u.sort(key=lambda t: t[1])
        for (a, _), (b, __) in zip(bends_for_u[:-1], bends_for_u[1:]):
            add_link(a, b)

    for leaf in leaves:
        add_node(f"{leaf}__tip", x_tip, y_of[leaf], "", "tip", COLOR_TIP)

    nodes_df = pd.DataFrame(node_rows, columns=["id", "x", "y", "label", "kind", "color"])
    links_df = pd.DataFrame(link_rows, columns=["source", "target"])

    nodes_df["x"] = nodes_df["x"].astype("float32")
    nodes_df["y"] = nodes_df["y"].astype("float32")
    for col in ("id", "label", "kind", "color"):
        nodes_df[col] = nodes_df[col].astype(str)
    links_df["source"] = links_df["source"].astype(str)
    links_df["target"] = links_df["target"].astype(str)

    log(f"[Layout] real={sum(nodes_df.kind.isin(['leaf','internal'])):,}, "
        f"bends={sum(nodes_df.kind == 'bend'):,}, tips={sum(nodes_df.kind == 'tip'):,}, "
        f"links={len(links_df):,}, xmax={nodes_df['x'].max():.3f}, ymax={nodes_df['y'].max():.3f}")

    return LayoutResult(nodes_df, links_df)


# 5) Build nodes/links for all trees in a file (ID-based)


In [5]:
def build_tables_for_input(path_or_text: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    all_nodes, all_links = [], []

    for name, newick in iter_newick_streams(path_or_text):
        log(f"[Build] parsing {name} ...")
        nodes = parse_newick(newick)
        layout = layout_weighted_rectangular(nodes)

        # If there are multiple trees, prefix IDs to keep them disjoint
        if name != "tree0":
            pref = name + ":"
            dfN = layout.nodes_df.copy()
            dfL = layout.links_df.copy()
            dfN["id"]    = pref + dfN["id"].astype(str)
            dfN["label"] = dfN["label"].astype(str).where(dfN["label"].astype(str) == "", pref + dfN["label"].astype(str))
            dfL["source"] = pref + dfL["source"].astype(str)
            dfL["target"] = pref + dfL["target"].astype(str)
        else:
            dfN, dfL = layout.nodes_df, layout.links_df

        all_nodes.append(dfN)
        all_links.append(dfL)

    NODES = pd.concat(all_nodes, ignore_index=True)
    LINKS = pd.concat(all_links, ignore_index=True)

    # Safety: drop links to missing nodes and self-loops
    idset = set(NODES["id"].astype(str))
    LINKS = LINKS[
        LINKS["source"].isin(idset) & LINKS["target"].isin(idset) & (LINKS["source"] != LINKS["target"])
    ].reset_index(drop=True)

    log(f"[Tables] nodes={len(NODES):,}, links={len(LINKS):,}")
    return NODES, LINKS

NODES_ID, LINKS_ID = build_tables_for_input(INPUT_PATH)


[Build] parsing tree0 ...
[Parse] nodes=2,480,482, leaves=2,239,042
[Layout] real=2,480,482, bends=4,960,962, tips=2,239,042, links=9,680,484, xmax=78.000, ymax=8956164.000
[Tables] nodes=9,680,486, links=9,680,484


# 6) Schema for `cosmograph_widget` (safe id-based mapping + validations)

In [6]:
def to_widget_points_links(nodes_df: pd.DataFrame, links_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Cosmograph widget accepts DataFrames and explicit column names (id/x/y and source/target).
    We keep an id-based schema to avoid index mismatches (common source of 'getChild' errors).
    Validates coordinates and endpoints strictly.
    """
    # points
    pts = nodes_df[["id", "x", "y", "label", "color"]].copy()
    pts = pts.dropna(subset=["x", "y"]).reset_index(drop=True)
    pts["id"]    = pts["id"].astype(str)
    pts["label"] = pts["label"].fillna("").astype(str)
    pts["color"] = pts["color"].fillna("").astype(str)
    pts["x"]     = pd.to_numeric(pts["x"], errors="coerce").astype("float64")
    pts["y"]     = pd.to_numeric(pts["y"], errors="coerce").astype("float64")

    # links
    lks = links_df[["source", "target"]].copy()
    lks["source"] = lks["source"].astype(str)
    lks["target"] = lks["target"].astype(str)

    # validations
    if pts[["x", "y"]].isna().any().any():
        raise AssertionError("[Schema] NaN in point coordinates.")
    pids = set(pts["id"].astype(str))
    bad = (~lks["source"].isin(pids)) | (~lks["target"].isin(pids)) | (lks["source"] == lks["target"])
    if bad.any():
        n_bad = int(bad.sum())
        log(f"[Schema] Dropping {n_bad} invalid links (dangling/self).")
        lks = lks[~bad].reset_index(drop=True)

    log(f"[Schema] points={pts.shape}, links={lks.shape}")
    return pts, lks

POINTS, LINKS = to_widget_points_links(NODES_ID, LINKS_ID)


[Schema] points=(9680486, 5), links=(9680484, 2)


# 7) Render with `cosmograph_widget` (static layout, visible colors)

In [7]:
# Uses the documented widget API with explicit mappings (id/x/y for points; source/target for links).
# Ref: Cosmograph Python widget configuration page (point_id_by, point_x_by, point_y_by, link_source_by, link_target_by).
# (If your environment has multiple versions installed, these field names are the stable entry points.)
# Docs: https://cosmograph.app/docs/cosmograph/Cosmograph%20Python/configuration/

try:
    from cosmograph_widget import Cosmograph
except Exception as e:
    log("[Render] cosmograph_widget is not importable. Run 'pip install cosmograph_widget'.")
    raise

def render_graph(points: pd.DataFrame, links: pd.DataFrame):
    log("[Render] creating widget ...")
    log(f"[Render] points columns: {list(points.columns)}")
    log(f"[Render] links columns: {list(links.columns)}")
    w = Cosmograph(
        points=points,
        links=links,
        # ---- point mapping ----
        point_id_by="id",
        point_x_by="x",
        point_y_by="y",
        point_label_by="label",
        point_color_by="color",
        # ---- link mapping ----
        link_source_by="source",
        link_target_by="target",
        # ---- appearance ----
        pointSize=POINT_SIZE,
        linkWidth=LINK_WIDTH,
        backgroundColor="#222222",   # default dark
        renderLinks=True,
        # ---- interaction/show ----
        showLabels=STATIC_LABELS,
        showDynamicLabels=DYNAMIC_LABELS,
        showHoveredPointLabel=HOVER_LABELS,
        # ---- IMPORTANT: we already computed positions; keep them static ----
        disableSimulation=True,
        enableDrag=True,
        fitViewOnInit=True,
        fitViewPadding=0.08,
    )
    display(w)
    try:
        w.fit_view()
    except Exception:
        pass
    log("[Render] done.")
    return w

WIDGET = render_graph(POINTS, LINKS)


[Render] creating widget ...


Cosmograph(background_color=None, focused_point_ring_color=None, hovered_point_ring_color=None, link_color=Non…

[Render] done.


# 8) Quick stats & structural checks (debug)

In [8]:
def quick_stats(nodes_df: pd.DataFrame, links_df: pd.DataFrame):
    kinds = nodes_df["kind"].value_counts(dropna=False)
    leaves = nodes_df[nodes_df["kind"] == "leaf"]
    tips   = nodes_df[nodes_df["kind"] == "tip"]

    log("[Stats] kinds:\n" + kinds.to_string())
    if len(leaves):
        xs = leaves["x"].to_numpy()
        log(f"[Stats] leaves X range: {xs.min():.3f} .. {xs.max():.3f} (varies with weights)")
    if len(tips):
        xt = float(tips["x"].iloc[0])
        log(f"[Stats] tip vertical line x = {xt:.3f}  (unconnected markers aligned)")
    log(f"[Stats] links: {len(links_df):,}")

quick_stats(NODES_ID, LINKS_ID)


[Stats] kinds:
kind
bend        4960962
leaf        2239042
tip         2239042
internal     241440
[Stats] leaves X range: 0.000 .. 58.000 (varies with weights)
[Stats] tip vertical line x = 78.000  (unconnected markers aligned)
[Stats] links: 9,680,484


# 9) One-shot runner `run_all(...)` (parses → lays out → renders)

In [9]:
def run_all(path_or_text: str = INPUT_PATH,
            leaf_step: float = LEAF_Y_STEP,
            parent_stub: float = PARENT_STUB_LEN,
            tip_pad: float = TIP_ALIGN_PAD):
    """
    Full pipeline with debug logs.
    Returns (points_df, links_df, widget).
    """
    log("[Run] starting ...")
    preview_input(path_or_text)

    all_nodes, all_links = [], []
    for name, newick in iter_newick_streams(path_or_text):
        log(f"[Run] tree={name} parse ...")
        nodes = parse_newick(newick)
        log(f"[Run] tree={name} layout ...")
        layout = layout_weighted_rectangular(nodes, leaf_step=leaf_step, parent_stub=parent_stub, tip_pad=tip_pad)

        if name != "tree0":
            pref = name + ":"
            dfN = layout.nodes_df.copy()
            dfL = layout.links_df.copy()
            dfN["id"]    = pref + dfN["id"].astype(str)
            dfN["label"] = dfN["label"].astype(str).where(dfN["label"].astype(str) == "", pref + dfN["label"].astype(str))
            dfL["source"] = pref + dfL["source"].astype(str)
            dfL["target"] = pref + dfL["target"].astype(str)
        else:
            dfN, dfL = layout.nodes_df, layout.links_df

        all_nodes.append(dfN)
        all_links.append(dfL)

    NODES = pd.concat(all_nodes, ignore_index=True)
    LINKS = pd.concat(all_links, ignore_index=True)

    log("[Run] schema step ...")
    POINTS, LKS = to_widget_points_links(NODES, LINKS)

    log("[Run] render ...")
    W = render_graph(POINTS, LKS)

    log("[Run] done.")
    return POINTS, LKS, W

# >>> Uncomment to run immediately:
# _POINTS, _LINKS, _W = run_all(INPUT_PATH)


In [10]:
run_all()

[Run] starting ...
[Input] file='/Users/gushchin_a/PyCharmProjects/Gtol/opentree15.1_tree/labelled_supertree/labelled_supertree_ottnames.tre'  size=85.83 MB  preview(len=400)


((((((((((((((((((((((((((((((((((((((((((((Elaeocarpus_williamsianus_ott2,((Platytheca_galioides_ott4,Platytheca_verticillata_ott414964,Platytheca_anasima_ott6107258,Platytheca_juniperina_ott6107259)Platytheca_ott414967,((((((((((((Elaeocarpus_foveolatus_ott6,(Elaeocarpus_reticulatus_ott22687,Elaeocarpus_bancroftii_ott471450)mrcaott22687ott471450)mrcaott6ott22687,Elaeocarpus_kirtonii_ott16)mrcaot


[Run] tree=tree0 parse ...
[Parse] nodes=2,480,482, leaves=2,239,042
[Run] tree=tree0 layout ...
[Layout] real=2,480,482, bends=4,960,962, tips=2,239,042, links=9,680,484, xmax=78.000, ymax=8956164.000
[Run] schema step ...
[Schema] points=(9680486, 5), links=(9680484, 2)
[Run] render ...
[Render] creating widget ...


Cosmograph(background_color=None, focused_point_ring_color=None, hovered_point_ring_color=None, link_color=Non…

[Render] done.
[Run] done.


(                    id     x           y                        label    color
 0                   n1   0.0  3230340.25  cellular_organisms_ott93302  #8ab4f8
 1                   n2   0.0  8767368.00          Eukaryota_ott304358  #8ab4f8
 2                   n3   0.0  8919577.00              mrcaott2ott3973  #8ab4f8
 3                   n4   0.0  8883243.00               mrcaott2ott276  #8ab4f8
 4                   n5   0.0  8818150.00               mrcaott2ott148  #8ab4f8
 ...                ...   ...         ...                          ...      ...
 9680481  n2480478__tip  78.0   466688.00                               #bdbdbd
 9680482  n2480479__tip  78.0   466680.00                               #bdbdbd
 9680483  n2480480__tip  78.0   466684.00                               #bdbdbd
 9680484  n2480481__tip  78.0   466752.00                               #bdbdbd
 9680485  n2480482__tip  78.0   466748.00                               #bdbdbd
 
 [9680486 rows x 5 columns],
         